In [1]:
import faiss
import pandas as pd
import datetime as dt
import numpy as np
from tqdm.notebook import tqdm

In [2]:
path = r"C:\Users\Admin\Documents\УЧЕБА\20 мастерская 2\notebooks\datasets\base_2.0.csv"

In [3]:
start_time = dt.datetime.now().timestamp()
base_2_0 = pd.read_csv(path, index_col=0)
dt.datetime.now().timestamp() - start_time

4.430881023406982

In [4]:
base_2_0.head(2)

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,class
Id,,,,,,,,,,,,,,,,,,,,,
1-base,-34.562202,13.332763,-69.78761,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.93793,116.83400,-84.888941,-59.524610,2.0
15-base,-133.391540,4.454095,-127.94336,-123.74695,143.213870,-38.265465,-224.939111,-84.442410,127.770160,50.926636,...,54.980003,-242.59491,103.561070,70.54476,-54.223568,142.79533,93.46864,-1074.464888,-108.886154,2.0


### Create index

In [5]:
start_time = dt.datetime.now().timestamp()

dims = base_2_0.shape[1]
n_cells = 1
# how many cells
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

dt.datetime.now().timestamp() - start_time

0.005557060241699219

In [6]:
start_time = dt.datetime.now().timestamp()

idx_l2.train(np.ascontiguousarray(base_2_0.values).astype('float32'))

idx_l2.add(np.ascontiguousarray(base_2_0.values).astype('float32'))
dt.datetime.now().timestamp() - start_time

0.716101884841919

In [7]:
base_index = {k: v for k, v in enumerate(base_2_0.index.to_list())}
len(base_index)

632110

# Train

In [8]:
df_train = pd.read_csv("datasets/train_2.0.csv", index_col=0)

In [9]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [10]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,class
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.51749,-87.144930,-347.360606,38.307602,109.08556,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.76360,-415.750254,-25.958572,2.0
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.80333,-83.810590,-94.572749,-78.433090,124.91590,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.58963,-1035.092211,-51.276833,2.0
5-query,-92.844185,2.975510,-61.760483,-171.67546,144.79837,-58.685143,104.112909,75.844580,118.33623,81.981125,...,12.807585,-137.96362,86.282088,64.678535,64.527600,64.664440,126.91460,-800.428664,-30.197390,2.0
7-query,-59.506752,7.959120,21.068153,-142.99788,128.15799,-92.496300,-143.453758,10.710561,131.08679,71.491560,...,49.263610,-203.78833,-101.989379,67.291770,44.437595,45.183838,150.28853,-1074.464888,-66.052086,2.0
10-query,-80.497215,4.198283,22.420013,-135.29889,119.97193,-81.749275,-530.124724,117.829230,128.16780,129.974720,...,41.703710,-192.11499,149.374627,67.501175,32.233040,86.700900,121.77761,-640.529192,-60.236816,2.0


In [11]:
start_time = dt.datetime.now().timestamp()
vecs, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), 50_000)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent, time_spent // 60, time_spent % 60

(55.00777316093445, 0.0, 55.00777316093445)

In [ ]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))